In [3]:
# pip install bitsandbytes -> uv 설치 에러남 
# pip install accelerate
import os
import json
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig

# -----------------------------
# 1. 모델 로드
# -----------------------------
MODEL_REPO = "Qwen/Qwen2-VL-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📌 Using device: {device}")

processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)

# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# -----------------------------
# 2. JSON 변환 함수
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    image = Image.open(image_path).convert("RGB")
    prompt = (
        "아래는 OCR로 추출된 텍스트 목록입니다. "
        "이 텍스트를 분석해서 key-value 형태의 JSON으로 만들어 주세요.\n"
        f"OCR 텍스트: {ocr_texts}\n\n"
        "1. key는 텍스트 의미에 맞게 추론\n"
        "2. value는 원문 그대로 유지\n"
        "출력 형식은 JSON만 반환"
    )

    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    result_text = processor.decode(outputs[0], skip_special_tokens=True)

    try:
        return json.loads(result_text)
    except json.JSONDecodeError:
        return {"result_text": result_text}

# -----------------------------
# 3. main 실행
# -----------------------------
if __name__ == "__main__":
    image_path = r"C:\Potenup\Drug-Detection-Chatbot\data\medicine_00451.jpeg"

    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)

    json_result = texts_and_image_to_json(image_path, ocr_texts)
    print("📌 LLM JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))

    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)

    print(f"✅ JSON 파일 저장 완료: {output_file}")

📌 Using device: cuda


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

PackageNotFoundError: No package metadata was found for bitsandbytes

In [2]:
import torch
import bitsandbytes as bnb

print("✅ PyTorch CUDA 버전:", torch.version.cuda)
print("✅ GPU 사용 가능 여부:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ 현재 GPU 이름:", torch.cuda.get_device_name(0))

try:
    from bitsandbytes.nn import Linear4bit
    print("✅ bitsandbytes 4bit 레이어 불러오기 성공")

    # 간단 테스트 (GPU 올려보기)
    lin = Linear4bit(16, 16).cuda()
    x = torch.randn(2, 16).cuda()
    y = lin(x)
    print("🚀 4bit 연산 성공, 출력 shape:", y.shape)
except Exception as e:
    print("❌ bitsandbytes 테스트 실패:", e)


ModuleNotFoundError: No module named 'bitsandbytes'

In [2]:
import os
import json
from PIL import Image
import torch
from drugocr import extract_text
from transformers import AutoProcessor, AutoModelForVision2Seq

# -----------------------------
# 1. 모델 불러오기
# -----------------------------
MODEL_REPO = "Qwen/Qwen2-VL-7B-Instruct"
device = "cpu"  # GPU 사용 가능하면 "cuda"로 변경
print(f"📌 Using device: {device}")

processor = AutoProcessor.from_pretrained(MODEL_REPO, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_REPO,
    trust_remote_code=True
).to(device)

# -----------------------------
# 2. OCR + 이미지 → JSON 변환 함수
# -----------------------------
def texts_and_image_to_json(image_path, ocr_texts):
    image = Image.open(image_path).convert("RGB")

    # Qwen2-VL 입력 프롬프트 (이미지 토큰 <image> 반드시 포함)
    prompt = (
        "아래는 OCR로 추출된 텍스트 목록입니다.\n"
        "<image>\n"  # Qwen2-VL은 반드시 이 토큰 필요
        f"OCR 텍스트: {ocr_texts}\n\n"
        "요구사항:\n"
        "1. key는 텍스트 의미에 맞게 추론\n"
        "2. value는 원문 그대로 유지\n"
        "3. 출력은 JSON만 반환"
    )

    # 입력 준비
    inputs = processor(
        text=prompt,
        images=[image],   # 반드시 리스트 형태
        return_tensors="pt"
    ).to(device)

    # 모델 실행
    outputs = model.generate(**inputs, max_new_tokens=512)
    result_text = processor.decode(outputs[0], skip_special_tokens=True)

    # JSON 변환 시도
    try:
        return json.loads(result_text)
    except json.JSONDecodeError:
        return {"result_text": result_text}

# -----------------------------
# 3. 실행부
# -----------------------------
if __name__ == "__main__":
    image_path = r"C:\Potenup\Drug-Detection-Chatbot\data\medicine_00451.jpeg"

    # OCR 추출
    ocr_texts = extract_text(image_path)
    print("📌 OCR 추출 결과:", ocr_texts)

    # 이미지+텍스트 → JSON 변환
    json_result = texts_and_image_to_json(image_path, ocr_texts)
    print("📌 LLM JSON 결과:")
    print(json.dumps(json_result, ensure_ascii=False, indent=2))

    # JSON 파일 저장
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"output_{base_name}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(json_result, f, ensure_ascii=False, indent=2)

    print(f"✅ JSON 파일 저장 완료: {output_file}")


c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\korean_PP-OCRv5_mobile_rec`.


📌 Using device: cpu


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\transformers\models\auto\modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

📌 OCR 추출 결과: ['[원료약품 및 그 분량]19 중', '·유효성분:', '디플루코르톨론발레레이트', '(BP) 3mg', '첨가제(보존제):', '파라옥시벤조산메틸(KP)', '1.8mg', '파라옥시벤조산프로필(KP)', '.0.2mg', '기타첨가제:', '경질유동파라핀,라우릴황산', '나트륨,모노스테아르산소르', '비탄,세탄올,스테아릴알코', '올,정제수,카보머940,트', '롤아민,프로필렌글리콜', '[성상]', '흰색~미담황색의 균질한 로션제', '[효능·효과]', '첨부문서참조', '[용법·용량]', '1일 2~3회 앞게  바른다.', '증상이 호전되면 1일 1회로', '충분하다.']


ValueError: Image features and image tokens do not match: tokens: 0, features 5719